# config

In [ ]:
#|default_exp config

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()
import repoyard.config as this_module

Environment variable DISABLE_NBLITE_EXPORT is set to True, skipping export.


In [ ]:
#|export
from pydantic import BaseModel, Field, model_validator
from typing import Literal
import os
from pathlib import Path
import toml, json
from enum import Enum

from repoyard import const

# `config.json`

In [ ]:
#|export
class StorageType(Enum):
    RCLONE = "rclone"
    LOCAL = "local"

class StorageConfig(const.StrictModel):
    storage_type: StorageType
    store_path: Path
    
    @model_validator(mode='after')
    def validate_config(self):
        # Expand paths
        self.store_path = self.store_path.expanduser()
        return self
    
class RepoGroupTitleMode(Enum):
    FULL_NAME = "full_name"
    DATETIME_AND_NAME = "datetime_and_name"
    NAME = "name"

class RepoGroupConfig(const.StrictModel):
    repo_title_mode: RepoGroupTitleMode = RepoGroupTitleMode.FULL_NAME
    unique_repo_names: bool = False

class VirtualRepoGroupConfig(const.StrictModel):
    repo_title_mode: RepoGroupTitleMode = RepoGroupTitleMode.FULL_NAME
    filter_expr: str

    def is_in_group(self, groups: list[str]) -> bool:
        if not hasattr(self, "_filter_func"):
            from repoyard._utils.logical_expressions import get_group_filter_func
            self._filter_func = get_group_filter_func(self.filter_expr)
        return self._filter_func(groups)

class Config(const.StrictModel):
    config_path : Path # Path to the config file. Will not be saved to the config file.
    
    default_storage_location : str
    repoyard_data_path : Path
    user_repos_path : Path
    user_repo_groups_path : Path
    storage_locations : dict[str, StorageConfig]
    repo_groups : dict[str, RepoGroupConfig]
    virtual_repo_groups : dict[str, VirtualRepoGroupConfig]
    default_repo_groups : list[str]
    repo_subid_character_set: str
    repo_subid_length: int
    max_concurrent_rclone_ops: int

    @property
    def local_store_path(self) -> Path:
        return self.repoyard_data_path / "local_store"

    @property
    def local_sync_backups_path(self) -> Path:
        return self.repoyard_data_path / "sync_backups"
    
    @property
    def repoyard_meta_path(self) -> Path:
        return self.repoyard_data_path / "repoyard_meta.json"
    
    @property
    def rclone_config_path(self) -> Path:
        return Path(self.config_path).parent / "repoyard_rclone.conf"
    
    @property
    def default_rclone_exclude_path(self) -> str:
        return self.config_path.parent / "default.rclone_exclude"
    
    @model_validator(mode='after')
    def validate_config(self):
        # Expand all paths
        self.config_path = Path(self.config_path).expanduser()
        self.repoyard_data_path = Path(self.repoyard_data_path).expanduser()
        self.user_repos_path = Path(self.user_repos_path).expanduser()
        self.user_repo_groups_path = Path(self.user_repo_groups_path).expanduser()
        
        import re
        for name in self.storage_locations.keys():
            if not re.fullmatch(r'[A-Za-z0-9_-]+', name):
                raise ValueError(f"StorageConfig name {name} is invalid. StorageConfig names can only contain alphanumeric characters, underscore(_), or dash(-).")
        
        if len(self.storage_locations) == 0:
            raise ValueError("No storage locations defined.")
            
        # Check that the default storage location exists
        if not any(name == self.default_storage_location for name in self.storage_locations):
            raise ValueError(f"default_storage_location '{self.default_storage_location}' not found in storage_locations")

        from repoyard._models import RepoMeta
        for group_name in list(self.repo_groups.keys()) + list(self.virtual_repo_groups.keys()):
            RepoMeta.validate_group_name(group_name)
        
        return self

In [ ]:
#|export
def get_config(path: Path|None = None) -> Config:
    if path is None: path = const.DEFAULT_CONFIG_PATH
    path = Path(path).expanduser()
    return Config(**{
        'config_path' : path,
        **toml.load(path)
    })

In [ ]:
#|export
def _get_default_config_dict(config_path=None, data_path=None) -> Config:
    if config_path is None: config_path = const.DEFAULT_CONFIG_PATH
    if data_path is None: data_path = const.DEFAULT_DATA_PATH
    config_path = Path(config_path)
    data_path = Path(data_path)
    
    config_dict = dict(
        config_path=config_path.as_posix(),
        default_storage_location = "fake",
        repoyard_data_path = data_path.as_posix(),
        user_repos_path = const.DEFAULT_USER_REPOS_PATH.as_posix(),
        user_repo_groups_path = const.DEFAULT_USER_REPO_GROUPS_PATH.as_posix(),
        storage_locations = {
            "fake": dict(
                storage_type=StorageType.LOCAL.value,
                store_path=(data_path / const.DEFAULT_FAKE_STORE_REL_PATH).as_posix(),
            )
        },
        repo_groups = {},
        virtual_repo_groups = {},
        default_repo_groups = [],
        repo_subid_character_set = const.DEFAULT_REPO_SUBID_CHARACTER_SET,
        repo_subid_length = const.DEFAULT_REPO_SUBID_LENGTH,
        max_concurrent_rclone_ops = const.DEFAULT_MAX_CONCURRENT_RCLONE_OPS,
    )
    return config_dict

# `rclone.conf`

In [ ]:
#|exporti
_default_rclone_config = """
"""